# 4-Bit Quantization bits&bytes

In [10]:
!pip install bitsandbytes

In [11]:
!pip install accelerate

In [12]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
import torch

In [13]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [14]:
model_name = "Writer/palmyra-med-20b"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

In [16]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config = bnb_config,
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
prompt = "A 21-year-old man presents to the emergency room complaining of pain upon urination and a watery discharge from his penis. It started a few days ago and has been getting progressively worse. His temperature is 98.0 F (36.7 C), blood pressure is 122/74 mmHg, pulse is 83/min, respirations are 14/min, and oxygen saturation is 98% on room air. Physical exam is notable for a tender urethra with a discharge. Gram stain of the discharge is negative for bacteria but shows many neutrophils. Patient's symptoms are the result of infection with Neisseria gonorrhoeae."

input_text = (
    "You're medical expert .Answer with the name of the disease only "
    "Only give name of disease based on following symptoms.If disease is already given then correct it."
    "Symptoms: {prompt} "
    "Disease/Diagonsis :"
)

model_inputs = tokenizer(input_text.format(prompt=prompt), return_tensors="pt").to(
    "cuda"
)

gen_conf = {
    "temperature": 0.7,
    "repetition_penalty": 1.0,
    "max_new_tokens": 512,
    "do_sample": True,
}

out_tokens = model.generate(**model_inputs, **gen_conf)

response_ids = out_tokens[0][len(model_inputs.input_ids[0]) :]
output = tokenizer.decode(response_ids, skip_special_tokens=True)

print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Gonorrhea Gonorrhea is a sexually transmitted infection caused by Neisseria gonorrhoeae. The bacteria can be found in the throat of a person who is infected, often producing a characteristic, foul-smelling discharge (also called gonorrhea). The bacteria can also be found in sexual secretions, including vaginal fluid, semen, and pre-ejaculate fluid. Transmission of the bacteria occurs through sexual contact (paicularly penis-to-anus sexual contact). It may also be transmitted through sexual contact with an infected person who has asymptomatic infection. N. gonorrhoeae infection can be treated with antibiotics. The antibiotics used are ceftriaxone, cefixime, ciprofloxacin, and azithromycin. The infection can lead to complications, including pelvic inflammatory disease, which increases the risk of ectopic pregnancy.


In [18]:
model.get_memory_footprint()


10800017496

# 8 Bit Quantisation

In [6]:
bnb_config_2 = BitsAndBytesConfig(
    load_in_8bit=True,  # Set to True for 8-bit quantization
    load_in_4bit=False,  # Set to False for 8-bit quantization
    llm_int8_threshold=0.5,  # Example threshold, adjust as needed
    llm_int8_skip_modules=None,  # You may specify modules to skip if needed
    llm_int8_enable_fp32_cpu_offload=False,  # You may enable FP32 CPU offload if needed
    llm_int8_has_fp16_weight=False,  # Specify if the model has FP16 weights
    bnb_4bit_compute_dtype=torch.uint8,  # Change dtype to uint8 for 8-bit quantization
    bnb_4bit_quant_type='nf8',  # Set to 'nf8' for 8-bit quantization
    bnb_4bit_use_double_quant=False  # You may specify double quantization if needed
)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config = bnb_config_2,
)

config.json:   0%|          | 0.00/797 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/39.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00005.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00005.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00003-of-00005.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00004-of-00005.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00005-of-00005.bin:   0%|          | 0.00/680M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
prompt = "A 21-year-old man presents to the emergency room complaining of pain upon urination and a watery discharge from his penis. It started a few days ago and has been getting progressively worse. His temperature is 98.0 F (36.7 C), blood pressure is 122/74 mmHg, pulse is 83/min, respirations are 14/min, and oxygen saturation is 98% on room air. Physical exam is notable for a tender urethra with a discharge. Gram stain of the discharge is negative for bacteria but shows many neutrophils. Patient's symptoms are the result of infection with Neisseria gonorrhoeae."

input_text = (
    "You're medical expert .Answer with the name of the disease only "
    "Only give name of disease based on following symptoms.If disease is already given then correct it."
    "Symptoms: {prompt} "
    "Disease/Diagonsis :"
)

model_inputs = tokenizer(input_text.format(prompt=prompt), return_tensors="pt").to(
    "cuda"
)

gen_conf = {
    "temperature": 0.7,
    "repetition_penalty": 1.0,
    "max_new_tokens": 512,
    "do_sample": True,
}

out_tokens = model.generate(**model_inputs, **gen_conf)

response_ids = out_tokens[0][len(model_inputs.input_ids[0]) :]
output = tokenizer.decode(response_ids, skip_special_tokens=True)

print(output)

# No Quantization

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Writer/palmyra-med-20b"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.int4,
)

In [ ]:
prompt = "Can you explain in simple terms how vaccines help our body fight diseases?"

input_text = (
    "A chat between a curious user and an artificial intelligence assistant. "
    "The assistant gives helpful, detailed, and polite answers to the user's questions. "
    "USER: {prompt} "
    "ASSISTANT:"
)

model_inputs = tokenizer(input_text.format(prompt=prompt), return_tensors="pt").to(
    "cuda"
)

gen_conf = {
    "temperature": 0.7,
    "repetition_penalty": 1.0,
    "max_new_tokens": 512,
    "do_sample": True,
}

out_tokens = model.generate(**model_inputs, **gen_conf)

response_ids = out_tokens[0][len(model_inputs.input_ids[0]) :]
output = tokenizer.decode(response_ids, skip_special_tokens=True)

print(output)


In [ ]:
prompt = "A 5-year-old male is brought to the pediatrician with complaints of a painful mouth/gums, and vesicular lesions on the lips and buccal mucosa for the past 4 days. The patient has not been able to eat or drink due to the pain and has been irritable. The patient also reports muscle aches. His vital signs are as follows: T 39.1, HR 110, BP 90/62, RR 18, SpO2 99%. Physical examination is significant for vesicular lesions noted on the tongue, gingiva, and lips, with some vesicles having ruptured and ulcerated, as well as palpable cervical and submandibular lymphadenopathy. Patient is diagnosed with ."

input_text = (
    "You're medical expert answer accordingly "
    "Only give name of disease based on following symptoms"
    "Symptoms: {prompt} "
    "Disease:"
)

model_inputs = tokenizer(input_text.format(prompt=prompt), return_tensors="pt").to(
    "cuda"
)

gen_conf = {
    "temperature": 0.7,
    "repetition_penalty": 1.0,
    "max_new_tokens": 512,
    "do_sample": True,
}

out_tokens = model.generate(**model_inputs, **gen_conf)

response_ids = out_tokens[0][len(model_inputs.input_ids[0]) :]
output = tokenizer.decode(response_ids, skip_special_tokens=True)

print(output)

In [ ]:

# input_text = (
#     f"You are medical expert. Only Return the disease you think is most probable based on following symptoms. Do NOT return anything else. Symptoms: {prompt}. The Patient is diagnoed with disease"
# )
input_txt = f"You are medical expert. Answer the name of disease you think is probable based on following symptoms: {prompt}"

In [ ]:
model_inputs = tokenizer(input_txt, return_tensors="pt").to(
    "cuda"
)

gen_conf = {
    "temperature": 0.7,
    "repetition_penalty": 1.0,
    "max_new_tokens": 512,
    "do_sample": True,
}

In [ ]:
out_tokens = model.generate(**model_inputs, **gen_conf)

In [ ]:
response_ids = out_tokens[0][len(model_inputs.input_ids[0]) :]
output = tokenizer.decode(response_ids, skip_special_tokens=True)

In [ ]:
output

In [ ]:
out_tokens

In [ ]:
# response_ids = out_tokens[0][len(model_inputs.input_ids[0]) :]
response_ids = out_tokens

In [ ]:
response_ids

In [ ]:
output = tokenizer.decode(response_ids, skip_special_tokens=True)

In [ ]:
print(output)